## 05. PyTorch Going Modular

#### => Turn useful notebook code cells into reusable Python scripts (.py files)
>
> e.g.: 
> - `data_setup.py` -> a file to prepare and download data if needed
> - `engine.py` -> a file containing various training functions
> - `model_builder.py` or `model.py` -> a file to create a PyTorch model
> - `train.py` -> a file to leverage all other files and train a target PyTorch model
> - `utils.py` -> a file dedicated to helpful utility functions

#### => Why go modular?
>
>  larger scale projects -> Python scripts more reproducible and easier to run.

#### How do I turn my notebook code into Python scripts?

**`Script mode:` uses `Jupyter Notebook cell magic` (special commands) to turn specific cells into Python scripts.**

Putting `%%writefile filename` at the top of a cell in Jupyter or Google Colab will write the contents of that cell to a specified filename.
> => e.g. 
>
> %%writefile hello_world.py
>
> print("hello world, machine learning is fun!")


### 0. Create a folder for storing Python scripts

In [1]:
import os
os.makedirs("go_modular", exist_ok=True)

### 1. Get Data

In [2]:
import os
import zipfile
from pathlib import Path
import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

data/pizza_steak_sushi directory exists.


In [3]:
# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Unzipping pizza, steak, sushi data...


In [4]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

### 2. Create Datasets and DataLoaders (script mode)

=> use the Jupyter magic function to create a `.py` file for creating DataLoaders.

=> save a code cell's contents to a file using the Jupyter magic `%%writefile filename`

In [5]:
%%writefile go_modular/data_setup.py

"""
contains functionality for creating PyTirch DataLoader's for image
classification data
"""

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size: int):
    # Use ImageFolder to create datasets(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes
    
    # Turn images into DataLoaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True 
    )

    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True 
    )

    return train_dataloader, test_dataloader, class_names

Writing go_modular/data_setup.py


In [6]:
from torchvision import transforms

# Create simple transform
data_transform = transforms.Compose([ 
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [7]:
from go_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x137e16bf0>,
 ['pizza', 'steak', 'sushi'])

### 3. Make a model with a script

In [8]:
%%writefile go_modular/model_builder.py

import torch
from torch import nn

class TinyVGG(nn.Module):
    """ model architecture copying TingVGG from CNN Explainer """
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
        self.con_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.con_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 15 * 15, out_features=output_shape)
        )
    
    def forward(self, x):
        x = self.con_block_1(x)
        x = self.con_block_2(x)
        x = self.classifier(x)
        return x

Writing go_modular/model_builder.py


In [9]:
import torch
from go_modular import model_builder

torch.manual_seed(42)

model_0 = model_builder.TinyVGG(input_shape=3,
                                hidden_units=10,
                                output_shape=len(class_names))
model_0

TinyVGG(
  (con_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (con_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2250, out_features=3, bias=True)
  )
)

In [10]:
# try a forward pass
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single)
    
# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0264, -0.0172, -0.0141]])

Output prediction probabilities:
tensor([[0.3427, 0.3281, 0.3291]])

Output prediction label:
tensor([0])

Actual label:
1
